In [10]:
import http.client
import json
import pandas as pd

def obter_dados_empresa_por_cnpj(cnpj):
    conexao = http.client.HTTPConnection("www.receitaws.com.br")
    conexao.request("GET",f"/v1/cnpj/{cnpj}")
    resposta = conexao.getresponse()
    print(f"Status da Resposta HTTP: {resposta.status}")
    
    if resposta.status != 200:
        return {"Status": "ERROR","Message":f"Resposta HTTP com status{resposta.status}"}
    
    dados = resposta.read()
    conexao.close()

    try:
        empresa = json.loads(dados.decode("utf-8"))
        print(f"Empresa decodificada: {empresa}")
        return empresa
    except json.JSONDecodeError as e:
        print(f"Erro na decodificação do JSON: {str(e)}")
        return {"Status": "ERROR","Message":"Error na decodificação do JSON"}
    

def salvar_dados_empresa_excel(empresa,nome_arquivo="dados_empresa.xlsx"):
    if empresa.get('status') != 'ERROR':
        empresa = tratar_dados_aninhados(empresa)
        df = pd.DataFrame([empresa])
        df.to_excel(nome_arquivo,index=False)
        print(f"Dados da empresa salvos com sucesso no arquivo {nome_arquivo}")
    else:
        print(f"Não há dados válidos para salvar. Mensagem de erro: {dados_empresa.get('message')}")

def tratar_dados_aninhados(dados):
    
    if "atividade_principal" in dados:
        dados['atividade_principal'] = "; ".join([ativ['text'] for ativ in dados['atividade_principal']])
    
    if "atividades_secundarias" in dados:
        dados['atividades_secundarias'] = "; ".join([ativ['text'] for ativ in dados['atividades_secundarias']])

    if 'qsa' in dados:
        dados['qsa'] = "; ".join([f"{q['nome']} ({q.get('qual','')})" for q in dados['qsa']])

    if 'billing' in dados:
        dados["billing"] = str(dados['billing'])

    if 'extra' in dados:
        dados["extra"] = str(dados['extra'])

    return dados

cnpj_exemplo = "33157312000162" 

dados_empresa = obter_dados_empresa_por_cnpj(cnpj_exemplo)

salvar_dados_empresa_excel(dados_empresa)

Status da Resposta HTTP: 200
Empresa decodificada: {'atividade_principal': [{'code': '82.99-7-02', 'text': 'Emissão de vales-alimentação, vales-transporte e similares'}], 'data_situacao': '27/03/2019', 'complemento': 'BLOCO B ANDAR 3 PARTE', 'tipo': 'MATRIZ', 'nome': 'IFOOD BENEFICIOS E SERVICOS LTDA.', 'telefone': '(11) 3634-3360', 'email': 'juridico@ifood.com.br', 'atividades_secundarias': [{'code': '82.99-7-99', 'text': 'Outras atividades de serviços prestados principalmente às empresas não especificadas anteriormente'}, {'code': '64.62-0-00', 'text': 'Holdings de instituições não-financeiras'}], 'qsa': [{'nome': 'FLAVIO DE FREITAS STECCA', 'qual': '05-Administrador'}, {'nome': 'LUCAS MARINI PITTIONI', 'qual': '05-Administrador'}, {'nome': 'IFOOD PARTICIPACOES LTDA.', 'qual': '22-Sócio', 'nome_rep_legal': 'DIEGO CARNEIRO BARRETO', 'qual_rep_legal': '17-Procurador'}, {'nome': 'GUSTAVO CARVALHO MENDES', 'qual': '05-Administrador'}, {'nome': 'DIEGO CARNEIRO BARRETO', 'qual': '05-Admini

In [38]:
import http.client
import json
import pandas as pd

def obter_dados_empresa_por_cnpj(cnpj):
    conexao = http.client.HTTPConnection("www.receitaws.com.br")
    conexao.request("GET",f"/v1/cnpj/{cnpj}")
    resposta = conexao.getresponse()
    print(f"Status da Resposta HTTP: {resposta.status}")
    
    if resposta.status != 200:
        print(f"Resposta HTTP com status{resposta.status}")
        return None
    
    dados = resposta.read()
    conexao.close()

    try:
        empresa = json.loads(dados.decode("utf-8"))
        print(f"Empresa decodificada: {empresa}")
        return empresa
    except json.JSONDecodeError as e:
        print(f"Erro na decodificação do JSON: {str(e)}")
        return None

caminho_planilha = "CNPJ.xlsx"
planhilha_cnpjs = pd.read_excel(caminho_planilha,sheet_name='CNPJ')
cnpjs = planhilha_cnpjs['CNPJ'].dropna()

resultados = pd.DataFrame(columns=['cnpj','nome','telefone','email',
'logradouro','bairro','municipio','uf','cep','atividade_principal'])

for cnpj in cnpjs:
    empresa = obter_dados_empresa_por_cnpj(str(cnpj).replace('-','').replace('/','').replace('.',''))
    if empresa:
        atividade_principal = ''
        if 'atividade_principal' in empresa and empresa['atividade_principal']:
            atividade_principal = empresa['atividade_principal'][0]['text']

        nova_linha = pd.DataFrame([{
            'cnpj': empresa.get('cnpj',''),
            'nome': empresa.get('nome',''),
            'telefone': empresa.get('telefone',''),
            'email': empresa.get('email',''),
            'logradouro': empresa.get('logradouro',''),
            'bairro': empresa.get('bairro',''),
            'municipio': empresa.get('municipio',''),
            'uf': empresa.get('uf',''),
            'cep': empresa.get('cep',''),
            'atividade_principal': atividade_principal
        }])
        resultados = pd.concat([resultados, nova_linha], ignore_index=True)

with pd.ExcelWriter(caminho_planilha, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    resultados.to_excel(writer, sheet_name='Dados', index=False)

print("Dados das empresas salvos na aba 'Dados' na planilha 'CNPJ.xlsx'")


Status da Resposta HTTP: 200
Empresa decodificada: {'atividade_principal': [{'code': '82.99-7-02', 'text': 'Emissão de vales-alimentação, vales-transporte e similares'}], 'data_situacao': '27/03/2019', 'complemento': 'BLOCO B ANDAR 3 PARTE', 'tipo': 'MATRIZ', 'nome': 'IFOOD BENEFICIOS E SERVICOS LTDA.', 'telefone': '(11) 3634-3360', 'email': 'juridico@ifood.com.br', 'atividades_secundarias': [{'code': '82.99-7-99', 'text': 'Outras atividades de serviços prestados principalmente às empresas não especificadas anteriormente'}, {'code': '64.62-0-00', 'text': 'Holdings de instituições não-financeiras'}], 'qsa': [{'nome': 'FLAVIO DE FREITAS STECCA', 'qual': '05-Administrador'}, {'nome': 'LUCAS MARINI PITTIONI', 'qual': '05-Administrador'}, {'nome': 'IFOOD PARTICIPACOES LTDA.', 'qual': '22-Sócio', 'nome_rep_legal': 'DIEGO CARNEIRO BARRETO', 'qual_rep_legal': '17-Procurador'}, {'nome': 'GUSTAVO CARVALHO MENDES', 'qual': '05-Administrador'}, {'nome': 'DIEGO CARNEIRO BARRETO', 'qual': '05-Admini